<a href="https://colab.research.google.com/github/MathiasGarnier/Algorithms-All---Old/blob/master/Fine_tuning_Qwen_Projet_EvaHan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# CHOISIR LE MODÈLE !!!
# Si on change de modèle, faut redémarrer la session colab / adapter son environnement
model_name = "unsloth/Qwen2.5-VL-7B-Instruct" # Compétition fermée
#model_name = "unsloth/Qwen3-VL-8B-Instruct" # Compétion ouverte, exemple de modèle (on peut monter à plus de 8B)
# https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen3_VL_(8B)-Vision.ipynb

# Je me suis fait manger mes crédits au moment où j'essayais de faire tourner Qwen3 (code ne change pas ensuite a priori)

In [2]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

if "2.5" and "7B" in model_name:
    print("Modèle 2.5, compétition fermée")
    !pip install transformers==4.56.2
elif "Qwen3" in model_name:
    print("Modèle 3, compétion ouverte (exemple de modèle)")
    !pip install transformers==4.57.1
else:
    print("Problème modèle")
!pip install --no-deps trl==0.22.2

In [3]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [ # On ne s'en sert pas ici
    "unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit", # Llama 3.2 vision support
    "unsloth/Llama-3.2-11B-Vision-bnb-4bit",
    "unsloth/Llama-3.2-90B-Vision-Instruct-bnb-4bit", # Can fit in a 80GB card!
    "unsloth/Llama-3.2-90B-Vision-bnb-4bit",

    "unsloth/Pixtral-12B-2409-bnb-4bit",              # Pixtral fits in 16GB!
    "unsloth/Pixtral-12B-Base-2409-bnb-4bit",         # Pixtral base model

    "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit",          # Qwen2 VL support
    "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit",
    "unsloth/Qwen2-VL-72B-Instruct-bnb-4bit",

    "unsloth/llava-v1.6-mistral-7b-hf-bnb-4bit",      # Any Llava variant works!
    "unsloth/llava-1.5-7b-hf-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastVisionModel.from_pretrained(
    model_name,
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Qwen2_5_Vl patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.90G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [14]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 32,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407, # Changer ça même si ça change rien
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

RuntimeError: Unsloth: You already added LoRA adapters to your model!

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import json
import os

base_path = '/content/drive/MyDrive/EVAHAN/train_data/'

##dataset_files = ['Dataset_A.json', 'Dataset_B.json', 'Dataset_C.json']
dataset_files = ['Dataset_A.json', 'Dataset_C.json']
#dataset_files = ['Dataset_B.json']
combined_dataset = []

for filename in dataset_files:
    filepath = os.path.join(base_path, filename)
    with open(filepath, 'r') as f:
        data = json.load(f)
        combined_dataset.extend(data)

print(f"Successfully loaded and combined {len(dataset_files)} datasets.")
print(f"Total items in combined dataset: {len(combined_dataset)}")

Successfully loaded and combined 2 datasets.
Total items in combined dataset: 10000


In [7]:
from sklearn.model_selection import train_test_split


# ON SE RESTREINT À PEU DE % DU JEU DE DONNÉES pour tester des trucs
subset_size = 1.0
subset_data, _ = train_test_split(combined_dataset, test_size=1, random_state=42)

print(f"Using a {subset_size*100}% subset of the data: {len(subset_data)} samples")

#train_data, temp_data, test_data, val_data

train_data, temp_data = train_test_split(subset_data, test_size=0.25, random_state=42)

test_data, val_data = train_test_split(temp_data, test_size=0.4, random_state=42)

print(f"Training set size: {len(train_data)}")
print(f"Testing set size: {len(test_data)}")
print(f"Validation set size: {len(val_data)}")

Using a 100.0% subset of the data: 9999 samples
Training set size: 7499
Testing set size: 1500
Validation set size: 1000


In [8]:
import PIL.Image
print("PIL.Image imported.")

PIL.Image imported.


In [9]:
from tqdm import tqdm # héhé
import json #
import os
import cv2 # binarisation image?
import numpy as np

instruction = """Analyze the provided image of ancient Chinese texts.

**Task Guidelines:**
1. **Transcription:** Transcribe the characters into standard Traditional Chinese (Unicode). Do not modernize the grammar.
2. **Legibility:** - If a character is an archaic variant, use the standard Traditional Chinese equivalent.
    - If a character is completely illegible due to damage, use '□'.
3. **Output:** Return ONLY the JSON object below that can be loaded using the loads function.


{
  "transcription": "TEXT_HERE",
  "notes": "Brief notes on damage/layout"
}
"""
# 4. **Reading Order:** If there is text, the text is generally written vertically (top to bottom) and arranged in columns from right to left. Respect this strict reading order.

base_path = '/content/drive/MyDrive/EVAHAN/train_data/'


# Adapted convert_to_conversation function
def convert_to_conversation_new(sample):
    if "text" not in sample:
        return None

    image_path = os.path.join(base_path, sample["image_path"])
    # PEUT-ÊTRE MODIFIER ÇA ?
    try:
    #    # Original:
      image = PIL.Image.open(image_path).convert("RGB")
    #    # NEW: Load image using OpenCV
    #    image_cv2 = cv2.imread(image_path)
    #    if image_cv2 is None:
    #        print(f"Error: Could not load image {image_path}")
    #        return None

    #    # Convert to grayscale
    #    gray_image = cv2.cvtColor(image_cv2, cv2.COLOR_BGR2GRAY)

        # Apply adaptive thresholding (binarization) to remove shadows
        # ADAPTIVE_THRESH_GAUSSIAN_C: uses a gaussian weighted sum of neighborhood values
        # THRESH_BINARY: the type of thresholding applied
        # 255: max value to use with THRESH_BINARY
        # 11: block size (size of neighborhood to calculate threshold for)
        # 2: constant subtracted from the mean or weighted mean
    #    binarized_image = cv2.adaptiveThreshold(gray_image, 255,
    #                                             cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    #                                             cv2.THRESH_BINARY, 11, 2)

        # Convert OpenCV image (numpy array) back to PIL Image
     #   image = PIL.Image.fromarray(binarized_image).convert("RGB") # Ensure RGB for model input

    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

    assistant_response_dict = {
        "transcription": sample["text"],
        "notes": ""
    }
    assistant_response_json_string = json.dumps(assistant_response_dict, ensure_ascii=False) # ensure_ascii=False to preserve Chinese characters

    conversation = [
        { "role": "user",
          "content" : [
            {"type" : "text",  "text"  : instruction},
            {"type" : "image", "image" : image} ]
        },
        { "role" : "assistant",
          "content" : [
            {"type" : "text",  "text"  : assistant_response_json_string} ]
        },
    ]
    return { "messages" : conversation }

converted_train_dataset = [convert_to_conversation_new(sample) for sample in tqdm(train_data) if convert_to_conversation_new(sample) is not None]
converted_val_dataset = [convert_to_conversation_new(sample) for sample in tqdm(val_data) if convert_to_conversation_new(sample) is not None]

print(f"Converted training dataset size: {len(converted_train_dataset)}")
print(f"Converted validation dataset size: {len(converted_val_dataset)}")

 93%|█████████▎| 6999/7499 [32:52<02:12,  3.78it/s]

Error processing image /content/drive/MyDrive/EVAHAN/train_data/Dataset_C/c_0939.jpg: image file is truncated (20 bytes not processed)


100%|██████████| 1000/1000 [04:53<00:00,  3.41it/s]

Converted training dataset size: 7498
Converted validation dataset size: 1000


In [10]:
train_data[0]

{'image_path': 'Dataset_C/c_1888.jpg', 'text': '舞腰争細鶯迁色並黄低𡸁連草帶亂拂和梅香雨滯'}

In [11]:
converted_train_dataset[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Analyze the provided image of ancient Chinese texts.\n\n**Task Guidelines:**\n1. **Transcription:** Transcribe the characters into standard Traditional Chinese (Unicode). Do not modernize the grammar.\n2. **Legibility:** - If a character is an archaic variant, use the standard Traditional Chinese equivalent.\n    - If a character is completely illegible due to damage, use \'□\'.\n3. **Output:** Return ONLY the JSON object below that can be loaded using the loads function.\n\n\n{\n  "transcription": "TEXT_HERE",\n  "notes": "Brief notes on damage/layout"\n}\n'},
    {'type': 'image',
     'image': <PIL.Image.Image image mode=RGB size=1425x96>}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': '{"transcription": "舞腰争細鶯迁色並黄低𡸁連草帶亂拂和梅香雨滯", "notes": ""}'}]}]}

In [ ]:
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model) # Enable for training!

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = UnslothVisionDataCollator(model, tokenizer), # Must use!
    train_dataset = converted_train_dataset, # Updated to use the new training dataset
    eval_dataset = converted_val_dataset,    # Added for evaluation
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # Set num_train_epochs to 1 and max_steps to -1 to train for one full epoch.
        # Adjust these values based on desired training duration and dataset size.
        max_steps = -1,                    # Set to -1 to train for num_train_epochs
        num_train_epochs = 1,              # Train for 1 full epoch
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",     # For Weights and Biases

        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        max_length = 2048,
    ),
)

Unsloth: Model does not have a default image size - using 512


In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.161 GB.
7.33 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,498 | Num Epochs = 1 | Total steps = 938
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 51,521,536 of 8,343,688,192 (0.62% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,4.634300
2,4.605100
3,4.564100
4,4.223500
5,3.670100
6,3.224600
7,2.766500
8,2.367900
9,2.195100
10,1.901200


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3079.6763 seconds used for training.
51.33 minutes used for training.
Peak reserved memory = 7.506 GB.
Peak reserved memory for training = 0.176 GB.
Peak reserved memory % of max memory = 33.87 %.
Peak reserved memory for training % of max memory = 0.794 %.


In [ ]:
import os
import PIL.Image

FastVisionModel.for_inference(model) # Enable for inference!

test_example = test_data[0]

print(test_example['text'])

image_path = os.path.join(base_path, test_example["image_path"])
image = PIL.Image.open(image_path).convert("RGB")

#instruction_ = instruction

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

於其不可分不可同之間正在作者會
{"transcription": "於其不可，有所回之；間止在任者，會", "notes": ""}<|im_end|>


In [12]:
import os

!wget -O task_a_c_eva.py https://raw.githubusercontent.com/GoThereGit/EvaHan/refs/heads/main/task_a_c_eva.py
!wget -O task_b_eva.py https://raw.githubusercontent.com/GoThereGit/EvaHan/refs/heads/main/task_b_eva.py

print("Downloaded task_a_c_eva.py and task_b_eva.py")

--2026-02-05 12:28:06--  https://raw.githubusercontent.com/GoThereGit/EvaHan/refs/heads/main/task_a_c_eva.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14094 (14K) [text/plain]
Saving to: ‘task_a_c_eva.py’

task_a_c_eva.py     100%[===================>]  13.76K  --.-KB/s    in 0.001s  

2026-02-05 12:28:06 (16.9 MB/s) - ‘task_a_c_eva.py’ saved [14094/14094]

--2026-02-05 12:28:06--  https://raw.githubusercontent.com/GoThereGit/EvaHan/refs/heads/main/task_b_eva.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20275 (20K) [te

In [ ]:
FastVisionModel.for_inference(model) # Ensure model is in inference mode

import json
from tqdm import tqdm
import os
import PIL.Image
import sys

if "/content/" not in sys.path:
    sys.path.append("/content/")

if 'task_a_c_eva' in sys.modules:
    del sys.modules['task_a_c_eva']
if 'task_b_eva' in sys.modules:
    del sys.modules['task_b_eva']

from task_a_c_eva import calculate_char_metrics # Corrected import
# from task_b_eva import LayoutEvaluator PETITES MODIF A FAIRE LÀ

sum = 0
idx = 0

for i, test_example in tqdm(enumerate(test_data[:50]), total=50, desc="Performing inference and evaluation"):
    image_path = os.path.join(base_path, test_example["image_path"])
    try:
        image = PIL.Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        continue

    messages = [
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": instruction}
        ]}
    ]

    input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)

    inputs = tokenizer(
        image,
        input_text,
        add_special_tokens = False,
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 128,
                       use_cache = True, temperature = 1.5, min_p = 0.1)

    generated_text_tokens = outputs[0][len(inputs["input_ids"][0]):]
    model_output_raw = tokenizer.decode(generated_text_tokens, skip_special_tokens=True)

    model_output_cleaned = model_output_raw.replace("<|im_end|>", "").strip()
    predicted_transcription = "N/A (parsing error)"
    try:
        model_output_for_json = model_output_cleaned.replace('\n', '\\n')
        parsed_output = json.loads(model_output_for_json)
        predicted_transcription = parsed_output.get("transcription", "N/A (transcription key missing)")
    except json.JSONDecodeError as e:
        print(f"JSON decoding error for output: {model_output_cleaned} - {e}")
        predicted_transcription = f"JSON Error: {model_output_cleaned}"

    ground_truth_transcription = test_example.get("text", "N/A (ground truth missing)")

    print(f"\n--- Example {i+1} ---")
    print(f"Model Output:     {predicted_transcription}")
    print(f"Ground Truth:     {ground_truth_transcription}")

    dataset_name = image_path.split('/')[-2]
    if "Dataset_B" in dataset_name:

        print("Evaluation (Dataset B): Not applicable. Model outputs transcription, but ground truth is for layout detection.")
        # RÉADAPTER POUR DATASET B

    else:

        if ground_truth_transcription != "N/A (ground truth missing)":

            metrics_ac = calculate_char_metrics(ground_truth_transcription, predicted_transcription) # Call the correct function
            score_ac = metrics_ac.get("comprehensive_score", "N/A (score missing)") # Extract the score
            score_cer = metrics_ac.get("cer", "N/A (score missing)")
            score_precision = metrics_ac.get("precision", "N/A (score missing)")
            score_recall = metrics_ac.get("recall", "N/A (score missing)")
            score_f1 = metrics_ac.get("f1", "N/A (score missing)")
            score_ned = metrics_ac.get("ned", "N/A (score missing)")

            print(f"Evaluation (Dataset A/C): Comprehensive Score = {score_ac}")
            print(f"Precisely: CER={score_cer}, P={score_precision}, R={score_recall}, F1={score_f1}, ned={score_ned}")
            idx += 1
            sum += score_ac
        else:
            print("Evaluation (Dataset A/C): Skipping due to missing ground truth.")


print(f"Mean: {sum / idx}")

Performing inference and evaluation:   2%|▏         | 1/50 [00:04<03:29,  4.28s/it]


--- Example 1 ---
Model Output:     於其不可，又不回之。間止在近者，會
Ground Truth:     於其不可分不可同之間正在作者會
Evaluation (Dataset A/C): Comprehensive Score = 0.5267
Precisely: CER=0.5333, P=0.5882, R=0.6667, F1=0.625, ned=0.4706


Performing inference and evaluation:   4%|▍         | 2/50 [00:08<03:29,  4.36s/it]


--- Example 2 ---
Model Output:     命當死吾家人亦不可辱於俘汝盡殺之而後殺我忠明
Ground Truth:     分當死吾家人亦不可辱於俘汝盡殺之而後殺我忠叩
Evaluation (Dataset A/C): Comprehensive Score = 0.9091
Precisely: CER=0.0909, P=0.9091, R=0.9091, F1=0.9091, ned=0.0909


Performing inference and evaluation:   6%|▌         | 3/50 [00:13<03:29,  4.46s/it]

...


--- Example 48 ---
Model Output:     性情。嗣顯仁公主長孫善禮，趙頊
Ground Truth:     脂𮧯調𤓖。長公在柱後時，值寜
Evaluation (Dataset A/C): Comprehensive Score = 0.1038
Precisely: CER=0.9286, P=0.1333, R=0.1429, F1=0.1379, ned=0.8667


Performing inference and evaluation:  98%|█████████▊| 49/50 [03:44<00:04,  4.57s/it]


--- Example 49 ---
Model Output:     終雖稱孤惡始獄中未紿共談主
Ground Truth:     波羅蜜多秘宻蔵中所說法故世閒
Evaluation (Dataset A/C): Comprehensive Score = 0.0722
Precisely: CER=0.9286, P=0.0769, R=0.0714, F1=0.0741, ned=0.9286


Performing inference and evaluation: 100%|██████████| 50/50 [03:49<00:00,  4.59s/it]


--- Example 50 ---
Model Output:     曉諭吾民其深省之其熟慮之爲此合行
Ground Truth:     曉諭吾民其深省之其熟慮之爲此合行
Evaluation (Dataset A/C): Comprehensive Score = 1.0
Precisely: CER=0.0, P=1.0, R=1.0, F1=1.0, ned=0.0
Mean: 0.4812139999999999


In [ ]:
model.save_pretrained("lora_model_QWEN2.5_EVAHAN")
tokenizer.save_pretrained("lora_model_QWEN2.5_EVAHAN")

[]

In [ ]:
print(test_data[19]['text']) # si + de 10 exemples, ne pas oublier de changer le 9 !
print(json.loads(model_output_cleaned)['transcription'])


自咲尋花獨後期殘紅粘砌緑縈枝尋常日對猶多恨
曉諭吾民其深省之其熟慮之為此合行


===========================================

==========================


# Task
Refine the current model by investigating and resolving 'image file is truncated' errors, experimenting with image preprocessing (e.g., binarization with `cv2.adaptiveThreshold`), increasing training epochs, tuning LoRA hyperparameters (`r`, `lora_alpha`, `lora_dropout`), and adjusting the learning rate. Additionally, expand the evaluation to include more examples from the test set. Analyze "Dataset_B.json" to determine its suitability for the transcription task; if appropriate, include it in the training data, or adapt "task_b_eva.py" for its specific evaluation. Finally, summarize the impact of these changes on the model's comprehensive score and generalization ability.

## Address Data Quality and Preprocessing

### Subtask:
Investigate and resolve 'image file is truncated' errors, and experiment with image preprocessing (e.g., binarization with cv2.adaptiveThreshold).


**Reasoning**:
The subtask requires experimenting with image preprocessing, specifically binarization, by uncommenting the relevant OpenCV code and commenting out the original PIL image loading in the `convert_to_conversation_new` function. I will modify the cell `b2221e6e` to enable this preprocessing.



In [ ]:
from tqdm import tqdm # héhé
import json #
import os
import cv2 # binarisation image?
import numpy as np

instruction = """Analyze the provided image of ancient Chinese texts.

**Task Guidelines:**
1. **Transcription:** Transcribe the characters into standard Traditional Chinese (Unicode). Do not modernize the grammar.
2. **Legibility:** - If a character is an archaic variant, use the standard Traditional Chinese equivalent.
    - If a character is completely illegible due to damage, use '□'.
3. **Output:** Return ONLY the JSON object below that can be loaded using the loads function.


{
  "transcription": "TEXT_HERE",
  "notes": "Brief notes on damage/layout"
}
"""
# 4. **Reading Order:** If there is text, the text is generally written vertically (top to bottom) and arranged in columns from right to left. Respect this strict reading order.

base_path = '/content/drive/MyDrive/EVAHAN/train_data/'


# Adapted convert_to_conversation function
def convert_to_conversation_new(sample):
    if "text" not in sample:
        return None

    image_path = os.path.join(base_path, sample["image_path"])
    # PEUT-ÊTRE MODIFIER ÇA ?
    try:
    #    # Original:
    #  image = PIL.Image.open(image_path).convert("RGB")
        # NEW: Load image using OpenCV
        image_cv2 = cv2.imread(image_path)
        if image_cv2 is None:
            print(f"Error: Could not load image {image_path}")
            return None

        # Convert to grayscale
        gray_image = cv2.cvtColor(image_cv2, cv2.COLOR_BGR2GRAY)

        # Apply adaptive thresholding (binarization) to remove shadows
        # ADAPTIVE_THRESH_GAUSSIAN_C: uses a gaussian weighted sum of neighborhood values
        # THRESH_BINARY: the type of thresholding applied
        # 255: max value to use with THRESH_BINARY
        # 11: block size (size of neighborhood to calculate threshold for)
        # 2: constant subtracted from the mean or weighted mean
        binarized_image = cv2.adaptiveThreshold(gray_image, 255,
                                                 cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                                 cv2.THRESH_BINARY, 11, 2)

        # Convert OpenCV image (numpy array) back to PIL Image
        image = PIL.Image.fromarray(binarized_image).convert("RGB") # Ensure RGB for model input

    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

    assistant_response_dict = {
        "transcription": sample["text"],
        "notes": ""
    }
    assistant_response_json_string = json.dumps(assistant_response_dict, ensure_ascii=False) # ensure_ascii=False to preserve Chinese characters

    conversation = [
        { "role" : "user",
          "content" : [
            {"type" : "text",  "text"  : instruction},
            {"type" : "image", "image" : image} ]
        },
        { "role" : "assistant",
          "content" : [
            {"type" : "text",  "text"  : assistant_response_json_string} ]
        },
    ]
    return { "messages" : conversation }

converted_train_dataset = [convert_to_conversation_new(sample) for sample in tqdm(train_data) if convert_to_conversation_new(sample) is not None]
converted_val_dataset = [convert_to_conversation_new(sample) for sample in tqdm(val_data) if convert_to_conversation_new(sample) is not None]

print(f"Converted training dataset size: {len(converted_train_dataset)}")
print(f"Converted validation dataset size: {len(converted_val_dataset)}")

100%|██████████| 1000/1000 [00:17<00:00, 58.82it/s]

Converted training dataset size: 7499
Converted validation dataset size: 1000


**Reasoning**:
The previous code block correctly implemented the requested changes in cell `b2221e6e`. The image preprocessing step (binarization) has been applied, and the dataset conversion completed without 'image truncated' errors, indicating that the OpenCV image loading and error handling within the function are working as expected. Now that the data preprocessing is handled, the next logical step is to re-run the `trainer.train()` method to train the model with the updated image preprocessing, which is in cell `d30ea6e6`.



In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,498 | Num Epochs = 1 | Total steps = 938
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 51,521,536 of 8,343,688,192 (0.62% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.120800
2,0.108800
3,0.254000
4,0.274800
5,0.049700
6,0.134500
7,0.116700
8,0.040700
9,0.092200
10,0.097400


In [ ]:
FastVisionModel.for_inference(model) # Ensure model is in inference mode

import json
from tqdm import tqdm
import os
import PIL.Image
import sys

if "/content/" not in sys.path:
    sys.path.append("/content/")

if 'task_a_c_eva' in sys.modules:
    del sys.modules['task_a_c_eva']
if 'task_b_eva' in sys.modules:
    del sys.modules['task_b_eva']

from task_a_c_eva import calculate_char_metrics # Corrected import
# from task_b_eva import LayoutEvaluator PETITES MODIF A FAIRE LÀ

sum = 0
idx = 0

for i, test_example in tqdm(enumerate(test_data[:50]), total=50, desc="Performing inference and evaluation"):
    image_path = os.path.join(base_path, test_example["image_path"])
    try:
        image = PIL.Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        continue

    messages = [
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": instruction}
        ]}
    ]

    input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)

    inputs = tokenizer(
        image,
        input_text,
        add_special_tokens = False,
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 128,
                       use_cache = True, temperature = 1.5, min_p = 0.1)

    generated_text_tokens = outputs[0][len(inputs["input_ids"][0]):]
    model_output_raw = tokenizer.decode(generated_text_tokens, skip_special_tokens=True)

    model_output_cleaned = model_output_raw.replace("<|im_end|>", "").strip()
    predicted_transcription = "N/A (parsing error)"
    try:
        model_output_for_json = model_output_cleaned.replace('\n', '\\n')
        parsed_output = json.loads(model_output_for_json)
        predicted_transcription = parsed_output.get("transcription", "N/A (transcription key missing)")
    except json.JSONDecodeError as e:
        print(f"JSON decoding error for output: {model_output_cleaned} - {e}")
        predicted_transcription = f"JSON Error: {model_output_cleaned}"

    ground_truth_transcription = test_example.get("text", "N/A (ground truth missing)")

    print(f"\n--- Example {i+1} ---")
    print(f"Model Output:     {predicted_transcription}")
    print(f"Ground Truth:     {ground_truth_transcription}")

    dataset_name = image_path.split('/')[-2]
    if "Dataset_B" in dataset_name:

        print("Evaluation (Dataset B): Not applicable. Model outputs transcription, but ground truth is for layout detection.")
        # RÉADAPTER POUR DATASET B

    else:

        if ground_truth_transcription != "N/A (ground truth missing)":

            metrics_ac = calculate_char_metrics(ground_truth_transcription, predicted_transcription) # Call the correct function
            score_ac = metrics_ac.get("comprehensive_score", "N/A (score missing)") # Extract the score
            score_cer = metrics_ac.get("cer", "N/A (score missing)")
            score_precision = metrics_ac.get("precision", "N/A (score missing)")
            score_recall = metrics_ac.get("recall", "N/A (score missing)")
            score_f1 = metrics_ac.get("f1", "N/A (score missing)")
            score_ned = metrics_ac.get("ned", "N/A (score missing)")

            print(f"Evaluation (Dataset A/C): Comprehensive Score = {score_ac}")
            print(f"Precisely: CER={score_cer}, P={score_precision}, R={score_recall}, F1={score_f1}, ned={score_ned}")
            idx += 1
            sum += score_ac
        else:
            print("Evaluation (Dataset A/C): Skipping due to missing ground truth.")


print(f"Mean: {sum / idx}")

Performing inference and evaluation:   2%|▏         | 1/50 [00:03<03:02,  3.72s/it]


--- Example 1 ---
Model Output:     於其不可者不可以問正在止者會
Ground Truth:     於其不可分不可同之間正在作者會
Evaluation (Dataset A/C): Comprehensive Score = 0.6736
Precisely: CER=0.3333, P=0.7143, R=0.6667, F1=0.6897, ned=0.3333


Performing inference and evaluation:   4%|▍         | 2/50 [00:08<03:20,  4.17s/it]


--- Example 2 ---
Model Output:     公當死吾家人亦不可辱於俘汝盡殺之而後殺我忠門
Ground Truth:     分當死吾家人亦不可辱於俘汝盡殺之而後殺我忠叩
Evaluation (Dataset A/C): Comprehensive Score = 0.9091
Precisely: CER=0.0909, P=0.9091, R=0.9091, F1=0.9091, ned=0.0909


Performing inference and evaluation:   6%|▌         | 3/50 [00:12<03:26,  4.39s/it]

...


--- Example 48 ---
Model Output:     聖道開竅俾公出乎德華特述頌之
Ground Truth:     脂𮧯調𤓖。長公在柱後時，值寜
Evaluation (Dataset A/C): Comprehensive Score = 0.0214
Precisely: CER=1.0, P=0.0714, R=0.0714, F1=0.0714, ned=1.0


Performing inference and evaluation:  98%|█████████▊| 49/50 [03:27<00:04,  4.28s/it]


--- Example 49 ---
Model Output:     參選翰林院侍讀中太尉共奉判國
Ground Truth:     波羅蜜多秘宻蔵中所說法故世閒
Evaluation (Dataset A/C): Comprehensive Score = 0.0714
Precisely: CER=0.9286, P=0.0714, R=0.0714, F1=0.0714, ned=0.9286


Performing inference and evaluation: 100%|██████████| 50/50 [03:31<00:00,  4.23s/it]


--- Example 50 ---
Model Output:     曉諭吾民其深省之其熟慮之為此合行
Ground Truth:     曉諭吾民其深省之其熟慮之爲此合行
Evaluation (Dataset A/C): Comprehensive Score = 0.9375
Precisely: CER=0.0625, P=0.9375, R=0.9375, F1=0.9375, ned=0.0625
Mean: 0.47425999999999996


Putain c'est nul (en même temps c'était à prévoir... vu la gueule du jeu de données déjà binarisé)

# Task
Adjust LoRA hyperparameters to `r=32`, `lora_alpha=32`, `lora_dropout=0.05`, and set `learning_rate=1e-4` in the `SFTConfig` within cell `06b60066`. Then, retrain the model using cell `d30ea6e6`, and subsequently re-evaluate its performance on the test set by executing cell `e854e219`, capturing the comprehensive score, CER, precision, recall, F1, and NED for analysis.

## Adjust LoRA Hyperparameters and Learning Rate

### Subtask:
Modify the `SFTConfig` in cell `06b60066` to experiment with new LoRA hyperparameters (`r`, `lora_alpha`, `lora_dropout`) and a different `learning_rate`. For this iteration, we will set `r=32`, `lora_alpha=32`, `lora_dropout=0.05`, and `learning_rate=1e-4`.


**Reasoning**:
To implement the requested LoRA hyperparameter and learning rate adjustments, I will modify cell `06b60066` by updating the `r`, `lora_alpha`, and `lora_dropout` values in the `get_peft_model` function and the `learning_rate` in the `SFTConfig`.



In [17]:
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model) # Enable for training!

# différents pas testés (learning_rate) 2e-4, 5e-6. Essayer peut-être 5e-5?
# passer sur un r=32 ?!

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = UnslothVisionDataCollator(model, tokenizer), # Must use!
    train_dataset = converted_train_dataset, # Updated to use the new training dataset
    eval_dataset = converted_val_dataset,    # Added for evaluation
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # Set num_train_epochs to 1 and max_steps to -1 to train for one full epoch.
        # Adjust these values based on desired training duration and dataset size.
        max_steps = -1,                    # Set to -1 to train for num_train_epochs
        num_train_epochs = 1,              # Train for 1 full epoch
        learning_rate = 5e-6,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",     # For Weights and Biases

        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        max_length = 2048,
    ),
)


Unsloth: Model does not have a default image size - using 512


In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,498 | Num Epochs = 1 | Total steps = 938
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 51,521,536 of 8,343,688,192 (0.62% trained)


Step,Training Loss
1,0.148400
2,0.189500
3,0.346900
4,0.387700
5,0.120500
6,0.250700
7,0.237700
8,0.084000
9,0.204800
10,0.134200


In [19]:
import os
import PIL.Image

FastVisionModel.for_inference(model) # Enable for inference!

test_example = test_data[0]

print(test_example['text'])

image_path = os.path.join(base_path, test_example["image_path"])
image = PIL.Image.open(image_path).convert("RGB")

#instruction_ = instruction

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

於其不可分不可同之間正在作者會


UnboundLocalError: cannot access local variable 'tracer_output' where it is not associated with a value

In [20]:
FastVisionModel.for_inference(model) # Ensure model is in inference mode

import json
from tqdm import tqdm
import os
import PIL.Image
import sys

if "/content/" not in sys.path:
    sys.path.append("/content/")

if 'task_a_c_eva' in sys.modules:
    del sys.modules['task_a_c_eva']
if 'task_b_eva' in sys.modules:
    del sys.modules['task_b_eva']

from task_a_c_eva import calculate_char_metrics # Corrected import
# from task_b_eva import LayoutEvaluator PETITES MODIF A FAIRE LÀ

sum = 0
med_cer, med_precision, med_recall, med_f1, med_ned = 0, 0, 0, 0, 0
idx = 0

for i, test_example in tqdm(enumerate(test_data[:50]), total=50, desc="Performing inference and evaluation"):
    image_path = os.path.join(base_path, test_example["image_path"])
    try:
        image = PIL.Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        continue

    messages = [
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": instruction}
        ]}
    ]

    input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)

    inputs = tokenizer(
        image,
        input_text,
        add_special_tokens = False,
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 128,
                       use_cache = True, temperature = 1.5, min_p = 0.1)

    generated_text_tokens = outputs[0][len(inputs["input_ids"][0]):]
    model_output_raw = tokenizer.decode(generated_text_tokens, skip_special_tokens=True)

    model_output_cleaned = model_output_raw.replace("<|im_end|>", "").strip()
    predicted_transcription = "N/A (parsing error)"
    try:
        model_output_for_json = model_output_cleaned.replace('\n', '\\n')
        parsed_output = json.loads(model_output_for_json)
        predicted_transcription = parsed_output.get("transcription", "N/A (transcription key missing)")
    except json.JSONDecodeError as e:
        print(f"JSON decoding error for output: {model_output_cleaned} - {e}")
        predicted_transcription = f"JSON Error: {model_output_cleaned}"

    ground_truth_transcription = test_example.get("text", "N/A (ground truth missing)")

    print(f"\n--- Example {i+1} ---")
    print(f"Model Output:     {predicted_transcription}")
    print(f"Ground Truth:     {ground_truth_transcription}")

    dataset_name = image_path.split('/')[-2]
    if "Dataset_B" in dataset_name:

        print("Evaluation (Dataset B): Not applicable. Model outputs transcription, but ground truth is for layout detection.")
        # RÉADAPTER POUR DATASET B

    else:

        if ground_truth_transcription != "N/A (ground truth missing)":

            metrics_ac = calculate_char_metrics(ground_truth_transcription, predicted_transcription) # Call the correct function
            score_ac = metrics_ac.get("comprehensive_score", "N/A (score missing)") # Extract the score
            score_cer = metrics_ac.get("cer", "N/A (score missing)")
            score_precision = metrics_ac.get("precision", "N/A (score missing)")
            score_recall = metrics_ac.get("recall", "N/A (score missing)")
            score_f1 = metrics_ac.get("f1", "N/A (score missing)")
            score_ned = metrics_ac.get("ned", "N/A (score missing)")

            print(f"Evaluation (Dataset A/C): Comprehensive Score = {score_ac}")
            print(f"Precisely: CER={score_cer}, P={score_precision}, R={score_recall}, F1={score_f1}, ned={score_ned}")
            idx += 1

            sum += score_ac
            med_cer += score_cer
            med_precision += score_precision
            med_recall += score_recall
            med_f1 += score_f1
            med_ned += score_ned

        else:
            print("Evaluation (Dataset A/C): Skipping due to missing ground truth.")


print(f"Mean (comprehensive): {sum / idx}")
print(f"Precisely (mean): CER={med_cer / idx}, P={med_precision / idx}, R={med_recall / idx}, F1={med_f1 / idx}, ned={med_ned / idx}")

Performing inference and evaluation:   2%|▏         | 1/50 [00:19<16:07, 19.75s/it]


--- Example 1 ---
Model Output:     於其不可否而不回之問正在逆者會
Ground Truth:     於其不可分不可同之間正在作者會
Evaluation (Dataset A/C): Comprehensive Score = 0.62
Precisely: CER=0.4, P=0.6667, R=0.6667, F1=0.6667, ned=0.4


Performing inference and evaluation:   4%|▍         | 2/50 [00:24<08:45, 10.94s/it]


--- Example 2 ---
Model Output:     分當死吾家人亦不可辱於俘汝盡殺之前後殺我忠明
Ground Truth:     分當死吾家人亦不可辱於俘汝盡殺之而後殺我忠叩
Evaluation (Dataset A/C): Comprehensive Score = 0.9091
Precisely: CER=0.0909, P=0.9091, R=0.9091, F1=0.9091, ned=0.0909


Performing inference and evaluation:   6%|▌         | 3/50 [00:29<06:22,  8.13s/it]

...


--- Example 48 ---
Model Output:     應州刺史李公世壺詩與何頲
Ground Truth:     脂𮧯調𤓖。長公在柱後時，值寜
Evaluation (Dataset A/C): Comprehensive Score = 0.0731
Precisely: CER=0.9286, P=0.0833, R=0.0714, F1=0.0769, ned=0.9286


Performing inference and evaluation:  98%|█████████▊| 49/50 [03:49<00:04,  4.31s/it]


--- Example 49 ---
Model Output:     參理刑部院君柏龄中大夫諡光裕正五
Ground Truth:     波羅蜜多秘宻蔵中所說法故世閒
Evaluation (Dataset A/C): Comprehensive Score = -0.0032
Precisely: CER=1.0714, P=0.0625, R=0.0714, F1=0.0667, ned=0.9375


Performing inference and evaluation: 100%|██████████| 50/50 [03:53<00:00,  4.67s/it]


--- Example 50 ---
Model Output:     曉諭吾民其深省之其熟慮之為此合行
Ground Truth:     曉諭吾民其深省之其熟慮之爲此合行
Evaluation (Dataset A/C): Comprehensive Score = 0.9375
Precisely: CER=0.0625, P=0.9375, R=0.9375, F1=0.9375, ned=0.0625
Mean (comprehensive): 0.4516380000000001
Precisely (mean): CER=0.556546, P=0.46901999999999994, R=0.46207600000000015, F1=0.4651240000000001, ned=0.5481379999999999


In [ ]:
model.save_pretrained("lora_model_QWEN2.5_EVAHAN")
tokenizer.save_pretrained("lora_model_QWEN2.5_EVAHAN")